<a href="https://colab.research.google.com/github/jsansao/teic-20231/blob/main/TEIC_Licao33_BuscaSemantica_Faiss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook Colab: Introdução ao FAISS

Este notebook demonstra um fluxo de trabalho básico para usar o **FAISS** (Facebook AI Similarity Search), uma biblioteca para busca de similaridade e clusterização eficiente de vetores densos.

Vamos:
1.  Instalar as bibliotecas necessárias.
2.  Criar um conjunto de frases (nosso "corpus").
3.  Usar `sentence-transformers` para converter essas frases em vetores (embeddings).
4.  Construir um índice FAISS simples (`IndexFlatL2`).
5.  Adicionar nossos vetores ao índice.
6.  Realizar uma busca de similaridade com uma nova frase (query).
7.  (Bônus) Mostrar um índice mais complexo (`IndexIVFFlat`) que requer treinamento.

### Célula 1: Instalação

Primeiro, vamos instalar o `faiss-cpu` (para rodar na CPU) e o `sentence-transformers`.

In [1]:
!pip install faiss-cpu sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 65.5 MB/s eta 0:00:00


### Célula 2: Importações e Definição do Modelo

Vamos importar as bibliotecas e carregar um modelo pré-treinado da `sentence-transformers` para criar nossos embeddings. Usaremos o `all-MiniLM-L6-v2`, um modelo leve e eficaz.

In [2]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

# Carrega o modelo de embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

print("Modelo e bibliotecas carregados.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:86: UserWarning: 
Access to the secret `HF_TOKEN` has not been granted on this notebook.
You will not be requested again.
Please restart the session if you want to be prompted again.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Modelo e bibliotecas carregados.


### Célula 3: Criação do Corpus e Geração dos Embeddings

Vamos definir nosso conjunto de documentos (frases) e convertê-los em vetores.

In [3]:
# 1. Nosso corpus de documentos
corpus = [
    "O cachorro late alto no quintal.",
    "Gatos são animais muito independentes.",
    "A inteligência artificial está transformando o mundo rapidamente.",
    "FAISS é uma biblioteca otimizada para busca de vetores.",
    "Eu gosto de programar em Python e construir aplicações de IA.",
    "O céu está azul e o dia está ensolarado hoje.",
    "Como funciona a busca por similaridade em larga escala?",
    "Um cão é o melhor amigo do homem."
]

# 2. Gerar os embeddings (vetores)
corpus_embeddings = model.encode(corpus)

# 3. Verificar a dimensionalidade
# Precisamos saber o tamanho dos vetores para o FAISS
d = corpus_embeddings.shape[1]
print(f"Número de frases no corpus: {len(corpus)}")
print(f"Dimensionalidade dos embeddings (d): {d}")
print(f"Shape do array de embeddings: {corpus_embeddings.shape}")

# É importante garantir que os dados estejam em float32
corpus_embeddings = corpus_embeddings.astype('float32')

Número de frases no corpus: 8
Dimensionalidade dos embeddings (d): 384
Shape do array de embeddings: (8, 384)


### Célula 4: Exemplo 1 - Índice Simples (IndexFlatL2)

O `IndexFlatL2` é o índice mais básico. Ele simplesmente armazena todos os vetores e, durante a busca, compara exaustivamente o vetor de consulta com todos os vetores no índice usando a distância L2 (Euclidiana).

É 100% preciso, mas lento para datasets muito grandes.

In [4]:
# 1. Criar o índice
index_flat = faiss.IndexFlatL2(d)  # d é a dimensionalidade do vetor

# 2. Verificar se o índice já está "treinado" (IndexFlatL2 não precisa de treino)
print(f"Índice 'is_trained': {index_flat.is_trained}")

# 3. Adicionar os vetores ao índice
index_flat.add(corpus_embeddings)

# 4. Verificar quantos vetores estão no índice
print(f"Total de vetores no índice: {index_flat.ntotal}")

Índice 'is_trained': True
Total de vetores no índice: 8


### Célula 5: Buscando no Índice Simples

Agora, vamos fazer uma pergunta (query) e encontrar as frases mais similares no nosso corpus.

In [11]:
# 1. Definir a consulta (query)
query_texto = "Quem é o melhor amigo de uma pessoa?"

# 2. Converter a query em um vetor (embedding)
query_vector = model.encode([query_texto])
query_vector = query_vector.astype('float32')

# 3. Definir quantos vizinhos mais próximos (k) queremos encontrar
k = 3

# 4. Realizar a busca
# D = Distâncias (quanto menor, mais similar)
# I = Índices (as posições dos vetores no corpus original)
D, I = index_flat.search(query_vector, k)

# 5. Mostrar os resultados
print(f"--- Resultados para a query: '{query_texto}' ---")
print("\nÍndices (I):", I)
print("Distâncias (D):", D)

print("\nFrases mais similares:")
for i, (idx, dist) in enumerate(zip(I[0], D[0])):
    print(f"{i+1}. Frase: '{corpus[idx]}'")
    print(f"   Distância (L2): {dist:.4f}")

--- Resultados para a query: 'Pets?' ---

Índices (I): [[7 1 6]]
Distâncias (D): [[1.8003187 1.8153975 1.8176161]]

Frases mais similares:
1. Frase: 'Um cão é o melhor amigo do homem.'
   Distância (L2): 1.8003
2. Frase: 'Gatos são animais muito independentes.'
   Distância (L2): 1.8154
3. Frase: 'Como funciona a busca por similaridade em larga escala?'
   Distância (L2): 1.8176


### Célula 6: Exemplo 2 - Índice Complexo (IndexIVFFlat)

Para datasets grandes, a busca exaustiva (Flat) é muito lenta. Usamos índices aproximados.

O `IndexIVFFlat` funciona dividindo o espaço vetorial em *clusters* (células). Para buscar, ele primeiro descobre em quais clusters o vetor de consulta provavelmente está e só compara com os vetores *dentro* desses clusters.

Isso requer um passo de **treinamento** (usando k-means) para encontrar os centroides desses clusters.

In [12]:
# 1. Definir o número de clusters (células)
nlist = 2  # Para um dataset tão pequeno, 2 é suficiente. Em um caso real, seria > 100

# 2. Definir um "quantizador" (basicamente, um índice que usaremos para encontrar os clusters)
quantizer = faiss.IndexFlatL2(d)

# 3. Criar o índice IVFFlat
# (quantizador, dimensão, n_clusters, métrica)
index_ivf = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

# 4. TREINAR o índice
# Ele precisa ver alguns dados (nossos embeddings) para definir os clusters
print("Treinando o índice IVFFlat...")
index_ivf.train(corpus_embeddings)
print(f"Índice 'is_trained': {index_ivf.is_trained}")

# 5. Adicionar os vetores (mesmo após o treino)
index_ivf.add(corpus_embeddings)
print(f"Total de vetores no índice: {index_ivf.ntotal}")

Treinando o índice IVFFlat...
Índice 'is_trained': True
Total de vetores no índice: 8


### Célula 7: Buscando no Índice Complexo (IVF)

A busca é similar, mas podemos ajustar um parâmetro: `nprobe`.

`nprobe` define quantas células (clusters) o FAISS deve visitar durante a busca. Um `nprobe` maior = mais preciso, mas mais lento. Um `nprobe` menor = mais rápido, mas menos preciso.

In [21]:
# 1. Definir quantos clusters queremos "procurar"
index_ivf.nprobe = 1  # Procurar apenas no cluster mais próximo (mais rápido)

# 2. Fazer a mesma busca de antes
query_texto_2 = "O que é busca?"
query_vector_2 = model.encode([query_texto_2]).astype('float32')
k = 3

D_ivf, I_ivf = index_ivf.search(query_vector_2, k)

# 3. Mostrar os resultados
print(f"--- Resultados (IVFFlat) para a query: '{query_texto_2}' ---")
print("\nÍndices (I):", I_ivf)
print("Distâncias (D):", D_ivf)

print("\nFrases mais similares:")
for i, (idx, dist) in enumerate(zip(I_ivf[0], D_ivf[0])):
    print(f"{i+1}. Frase: '{corpus[idx]}'")
    print(f"   Distância (L2): {dist:.4f}")

--- Resultados (IVFFlat) para a query: 'O que é busca?' ---

Índices (I): [[6 3 5]]
Distâncias (D): [[0.66531694 0.8450552  0.99520624]]

Frases mais similares:
1. Frase: 'Como funciona a busca por similaridade em larga escala?'
   Distância (L2): 0.6653
2. Frase: 'FAISS é uma biblioteca otimizada para busca de vetores.'
   Distância (L2): 0.8451
3. Frase: 'O céu está azul e o dia está ensolarado hoje.'
   Distância (L2): 0.9952
